In [6]:
import numpy as np
from numpy import *

In [14]:
import pandas as pd
# Upload data.
df = pd.DataFrame(pd.read_csv('deathrate_instance_python.dat', header=None, sep="\s+"))
df.columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'Y']
# Define matrix X and vector Y.
Y = df['Y']
X = df.drop(columns=['Y'])
# Add a column of ones.
#X = pd.concat([X, pd.DataFrame({'O': np.ones(len(X))})], axis=1) 
# Transform dataframe into matrix.
Ymat = Y.values
Xmat = X.values

In [15]:
# Loss func loss(x)
def loss_func (x):
    """
    Objective function: loss / cost function.
    
    Parameters
    ----------
    w array of weights.
    gamma array of intercepts.
    
    Returns
    -------
    The objective function loss(x) evaluated at {w, gamma}.
    
    """
    w = np.asarray(x[:-1]).T
    gamma = x[-1]
    
    cost = (1/2) * ((Xmat @ w) + gamma - Ymat).T @ ((Xmat @ w) + gamma - Ymat)
  
    return np.squeeze(cost)

In [30]:
def loss_grad (x): 
    """
    Gradient of the objective function (loss / cost function).
    
    Parameters
    ----------
    A matrix with x arrays at the rows.
    w array of weights.
    gamma array of intercepts.
    y array of f(x).
    
    Returns
    -------
    Array of the gradient of the objective function loss(x) 
    evaluated at {w, gamma}.
    
    """
    w = np.asarray(x[:-1]).T
    gamma = x[-1]
    
    g_cost_comp1 = Xmat.T @ ((Xmat @ w) + gamma - Ymat)
    g_cost_comp2 = np.ones(len(Ymat)).T @ ((Xmat @ w) + gamma - Ymat)
    
    return np.concatenate((g_cost_comp1, g_cost_comp2), axis=None)

In [35]:
def loss_hess (x):
    """
    Hessian matrix of the objective function (loss / cost function).
    
    Parameters
    ----------
    w array of weights.
    gamma array of intercepts.
    
    Returns
    -------
    Matrix of the Hessian of the objective function loss(x) 
    evaluated at {w, gamma}.
    
    """
    w = np.asarray(x[:-1]).T
    gamma = x[-1]
    
    block_top_left = Xmat.T @ Xmat
    block_top_right = Xmat.T @ np.ones(len(Ymat))
    
    block_bottom_left = np.ones(len(Ymat)).T @ Xmat
    block_bottom_right = len(Ymat)
    
    H = np.empty([len(x), len(x)])
    
    H[:-1,:-1] = block_top_left
    H[:-1,-1] = block_top_right
    
    H[-1,:-1] = block_bottom_left
    H[-1,-1] = block_bottom_right
    
    return H

In [32]:
wopt = [0.48824,-0.0463881,0.102978,-0.0377061,0.00490736,-0.0339172,-0.255786,0.00564845,0.649708,-0.12622,0.213407,-0.207831,0.109842,0.376641,0.00995978,895.141]

In [33]:
loss_func(wopt)

61484.650716435746

In [29]:
loss_grad(wopt)

(15,)
(60, 15)


array([-1.05322859e+04,  9.99806306e+02, -2.22295893e+03,  8.13127974e+02,
       -1.05927484e+02,  7.31350810e+02,  5.51550115e+03, -2.10784096e+02,
       -1.40146068e+04,  2.72154380e+03, -4.60357165e+03,  4.48211610e+03,
       -2.36983501e+03, -8.12547179e+03, -2.16150552e+02, -2.28754456e-02])

In [37]:
# loss_hess(wopt)

In [38]:
def cons_func (x):
    """
    Constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The constraint evaluated at x.
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    return np.sum(w**2)

In [39]:
def cons_jacobian (x):
    """
    Gradient of the constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The gradient of the constraint evaluated at x.
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    return (2*w).tolist()

In [40]:
def cons_hess (x, v):
    """
    Product of the hessian of the constraint by an arbitrary vector.
    It is needed for the minimize function.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    v vector of length = number of constraints.
    
    Returns
    -------
    The Hessian of the constraint evaluated at x by v[0].
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    H = (np.eye(w.shape[0])*2)
    H[-1,-1] = 0
    return v[0] * H

In [42]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

# Define hyperparamater t:
t = 1

## OPTIMIZATION ##
# Assign initial random weights:
w = np.squeeze(np.random.rand(1,16))

# Define non linear constraint:
nonlinear_constraint = NonlinearConstraint(cons_func, -np.inf, t, jac = cons_jacobian, hess = cons_hess) 

# Solve:
sol = minimize(loss_func, wopt, method='trust-constr', jac=loss_grad, hess=loss_hess,
               constraints=[nonlinear_constraint], options={'verbose': 1})
# Output:
print(sol)

`xtol` termination condition is satisfied.
Number of iterations: 314, function evaluations: 456, CG iterations: 1051, optimality: 9.01e+07, constraint violation: 2.30e-02, execution time: 0.45 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 1051
      cg_stop_cond: 2
            constr: [array([1.02304613])]
       constr_nfev: [456]
       constr_nhev: [83]
       constr_njev: [82]
    constr_penalty: 1.0
  constr_violation: 0.02304613284172996
    execution_time: 0.4453599452972412
               fun: 580796.2044913888
              grad: array([-9.94626925e+05, -8.89928901e+05, -1.96002339e+06, -2.27754714e+05,
       -8.58390398e+04, -2.87610861e+05, -2.10429325e+06, -9.12069718e+07,
       -3.30456974e+05, -1.20311362e+06, -3.87587700e+05, -8.70050869e+05,
       -5.26238947e+05, -1.27704025e+06, -1.50967026e+06, -2.61943093e+04])
               jac: [array([[ 0.56804062,  0.18175717,  0.99391003,  0.1658011 ,  0.11150358,